In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import pickle
with open('/content/drive/My Drive/assignment/Assignment(Taxi_demand)/X_train_new','rb') as f:
    X_train_new = pickle.load(f)
f.close()

with open('/content/drive/My Drive/assignment/Assignment(Taxi_demand)/X_test_new','rb') as f:
    X_test_new = pickle.load(f)
f.close()

with open('/content/drive/My Drive/assignment/Assignment(Taxi_demand)/y_train','rb') as f:
    y_train = pickle.load(f)
f.close()

with open('/content/drive/My Drive/assignment/Assignment(Taxi_demand)/y_test','rb') as f:
    y_test = pickle.load(f)
f.close()

In [4]:
print(X_train_new.shape,len(y_train))
print(X_test_new.shape,len(y_train))

(366760, 68) 366760
(157200, 68) 366760


#**Xgboost Regressor**

In [0]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [6]:
params = {'max_depth':[3,4,5,6,7],
          'min_child_weight':[0.01,0.1,1,10],
          'n_estimators':[50,100,200,500]}

model  = xgb.XGBRegressor(booster='gbtree')
grid = RandomizedSearchCV(model, param_distributions=params, scoring= 'neg_mean_squared_error', \
                    n_jobs=-1,cv=5,return_train_score=True) 
                                                
grid.fit(X_train_new,y_train) 

[20:25:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=0,
                                          reg_lambda=1, scale_pos_weight=1,
                                          seed=None, silent=None, subsample=1,
                                  

In [7]:
grid.cv_results_

{'mean_fit_time': array([ 133.04944167,   89.8542202 ,  491.40316463, 1230.41535001,
        1006.10322976,  500.14832621,  109.81159258,   90.56573262,
        1220.3815556 ,  300.33195791]),
 'mean_score_time': array([0.35871582, 0.21148005, 0.95353117, 2.52146745, 1.95805216,
        0.9611496 , 0.25236025, 0.20698366, 2.6727695 , 0.64625688]),
 'mean_test_score': array([-165.13354652, -164.73327019, -165.81777777, -167.84648949,
        -166.36392965, -165.85319143, -164.43516508, -164.73327019,
        -167.84648949, -164.31571763]),
 'mean_train_score': array([-150.04227218, -157.3232763 , -139.70246023, -123.1034365 ,
        -133.75892379, -138.03338946, -154.39892641, -157.3232763 ,
        -123.1034365 , -150.43239839]),
 'param_max_depth': masked_array(data=[6, 4, 6, 6, 5, 6, 5, 4, 6, 4],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_min_child_weight':

In [9]:
grid.best_params_

{'max_depth': 4, 'min_child_weight': 10, 'n_estimators': 200}

In [10]:
grid.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=10, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [11]:
model = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=10, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

model.fit(X_train_new,y_train)
y_train_pred = model.predict(X_train_new)
y_test_pred = model.predict(X_test_new)

final_result_train = (mean_absolute_error(y_train,y_train_pred))/(sum(y_train)/len(y_train))
final_result_test = (mean_absolute_error(y_test,y_test_pred))/(sum(y_test)/len(y_test))

print("MAPE for training data : ",final_result_train)
print("MAPE for test data : ",final_result_test)

[20:51:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAPE for training data :  0.12870909850859635
MAPE for test data :  0.13171723884520675
